In [1]:
import warnings
warnings.filterwarnings("ignore")
from trainer import load_and_preprocess_data
from tool import utils as ul
from tool import config as cfg
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import xgboost as xgb

2024-03-05 07:53:46.922183: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 07:53:46.922222: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 07:53:46.922248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 07:53:46.931471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 07:53:48.344053: W tensorflow/c

In [21]:
cpx = pd.read_csv('./DATA/complexPortal_num(1).tsv',sep= '\t')
feather = pd.read_feather('./featurebank/seq_esm.feather')
dataset = pd.merge(feather,cpx,left_on="uniprot_id" ,right_on="entry")
dataset = dataset[dataset["comple_portal_num"].isin([1,2,3,4,5,6,7,8,9,10,12])]
# dataset["comple_portal_num"] = dataset["comple_portal_num"].replace(dict(zip([1,2,3,4,5,6,7,8,10,12],[0,1,2,3,4,5,6,7,8,9])))


In [22]:
X = np.array(dataset.iloc[:,1:-2])
y = dataset['comple_portal_num'].values
# Load and preprocess the dataset.
ground_truth_labels=[]
predicted_labels = []

In [24]:
with open('./model/ml_xg.pkl', 'rb') as file:
    xg_model = pickle.load(file)
xg_predict = xg_model.predict(X)

with open('./model/ml_knn.pkl', 'rb') as file:
    knn_model = pickle.load(file)
xg_predict = xg_model.predict(X)
knn_predict = knn_model.predict(X)

In [25]:
label_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12}
y_xg_transformed = [label_map[x] for x in xg_predict]
y_knn_transformed = [label_map[x] for x in knn_predict]


In [26]:
report = pd.DataFrame(classification_report(y, y_xg_transformed,output_dict=True)).T
report.to_csv("xg_report.csv")
display(report)

,precision,recall,f1-score,support
1,0.816456,0.185212,0.301931,2786.000000
2,0.212766,0.845638,0.339989,745.000000
3,0.485507,0.443709,0.463668,151.000000
4,0.436893,0.378151,0.405405,119.000000
5,0.538462,0.259259,0.350000,27.000000
6,0.268293,0.360656,0.307692,61.000000
7,0.000000,0.000000,0.000000,7.000000
8,1.000000,0.083333,0.153846,12.000000
9,0.000000,0.000000,0.000000,4.000000
10,1.000000,0.200000,0.333333,5.000000


In [27]:
report = pd.DataFrame(classification_report(y, y_knn_transformed,output_dict=True)).T
report.to_csv("knn_report.csv")
display(report)

,precision,recall,f1-score,support
1,0.767538,0.200287,0.317677,2786.000000
2,0.204590,0.706040,0.317250,745.000000
3,0.335052,0.430464,0.376812,151.000000
4,0.245810,0.369748,0.295302,119.000000
5,0.200000,0.407407,0.268293,27.000000
6,0.104478,0.344262,0.160305,61.000000
7,0.000000,0.000000,0.000000,7.000000
8,0.000000,0.000000,0.000000,12.000000
9,0.000000,0.000000,0.000000,4.000000
10,0.000000,0.000000,0.000000,5.000000


In [28]:
output_result = dataset.copy()
output_result = output_result[["uniprot_id","comple_portal_num"]]
output_result["XGBOOST_Predict"] = y_xg_transformed
output_result["KNN_Predict"] = y_knn_transformed

In [29]:
output_result

,uniprot_id,comple_portal_num,XGBOOST_Predict,KNN_Predict
0,Q01147,1,2,2
1,Q06507,1,2,2
2,P81269,1,2,2
3,P97471,1,1,1
4,P19123,1,2,1
...,...,...,...,...
3945,Q9TS33,4,4,2
3946,P07293,1,2,4
3947,P13806,1,2,2
3948,P19517,1,2,2


#### blast

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from tool import model as md
from tool.blast import getblast
from tool import config as cfg
from sklearn.metrics import classification_report

from test import run
import pandas as pd
import os


dataset = pd.read_csv(cfg.DATA_PATH)
train_data = dataset.rename(columns={'Entry':'id','Sequence':'seq'})

dataset = pd.read_csv("./output/download_seq.csv")
cpx = pd.read_csv('./DATA/complexPortal_num(1).tsv',sep= '\t')
dataset = pd.merge(dataset,cpx,left_on="uniprot_id" ,right_on="entry")
dataset = dataset[dataset["comple_portal_num"].isin([1,2,3,4,5,6,7,8,9,10,12])]
vali_data = dataset.rename(columns={'uniprot_id':'id','Sequence':'seq','comple_portal_num':'label'})
  
diamond_task = getblast(train = train_data, test = vali_data)
diamond_task

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org



diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 -e 1e-5 --quiet


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org



,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Q01147,Q9VWW0,84.7,59,9,0,265,323,296,354,1.800000e-20,99.0
1,P81269,Q9VWW0,82.5,57,10,0,208,264,297,353,3.000000e-18,91.3
2,P97471,Q13485,98.4,552,8,1,1,551,1,552,0.000000e+00,1102.4
3,P18293,P18910,98.2,1057,19,0,1,1057,1,1057,0.000000e+00,2099.7
4,P61014,P61014,100.0,52,0,0,1,52,1,52,1.300000e-15,80.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1267,Q9MZW3,P02459,51.7,87,33,1,2,88,12,89,4.800000e-22,103.6
1268,W5NTT7,P02460,65.3,251,84,2,1114,1363,610,858,1.800000e-98,360.1
1269,P11716,P11716,100.0,5037,0,0,1,5037,1,5037,0.000000e+00,9710.1
1270,Q9TS33,Q9TS33,100.0,4872,0,0,1,4872,1,4872,0.000000e+00,9697.0


In [31]:
# 按照Query列分组，取每个组中最相似的记录
best_matches_df = diamond_task.loc[diamond_task.groupby('id')['bitscore'].idxmax()]
best_matches_df

,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
772,A0A1D5P2B5,P19793,92.1,467,32,2,1,467,1,462,1.100000e-233,807.7
1249,A0A287A1S6,P02460,68.8,272,82,3,1197,1466,589,859,1.600000e-113,410.2
519,A0A2P9E102,Q6G1V5,35.0,234,133,6,8,233,7,229,8.100000e-27,119.8
1242,A0A8C0MYN9,P05539,68.4,234,74,0,1232,1465,1186,1419,5.700000e-103,375.2
1243,A0A8C0N620,P02460,60.6,259,100,2,1242,1499,602,859,2.200000e-89,330.1
...,...,...,...,...,...,...,...,...,...,...,...,...
253,Q9Z1T2,Q06441,76.0,958,221,6,4,958,2,953,0.000000e+00,1080.9
180,Q9Z202,Q9BXN2,26.4,208,120,5,66,242,40,245,3.500000e-13,74.3
122,Q9Z321,O28469,27.8,699,442,18,5,691,6,653,1.300000e-62,240.4
442,Q9ZZW7,P03880,46.4,519,245,4,1,517,1,488,3.100000e-128,457.6


In [32]:
label_merge = pd.merge(best_matches_df,train_data[['id','label']], left_on="sseqid", right_on="id")
label_merge = pd.merge(label_merge,vali_data[['id','label']], left_on="id_x", right_on="id")
result = label_merge[['id_x', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
       'qstart', 'qend', 'sstart', 'send', 'evalue',
       'bitscore','label_x', 'label_y']]
result.columns = ['Query', 'Subject', 'Identity', 'Length', 'Mismatches', 'Gaps',
       'Query_start', 'Query_end', 'Subject_start', 'Subject_end', 'E_value',
       'Bit_score','Predict', 'GroundTruth']
predict_miss = vali_data[vali_data.id.isin((set(vali_data.id)-set(result.Query)))][["id","label"]]
result

,Query,Subject,Identity,Length,Mismatches,Gaps,Query_start,Query_end,Subject_start,Subject_end,E_value,Bit_score,Predict,GroundTruth
0,A0A1D5P2B5,P19793,92.1,467,32,2,1,467,1,462,1.100000e-233,807.7,2,1
1,P10827,P19793,29.2,342,207,7,30,370,114,421,4.500000e-36,151.4,2,1
2,P63058,P19793,29.2,342,207,7,30,370,114,421,4.500000e-36,151.4,2,1
3,A0A287A1S6,P02460,68.8,272,82,3,1197,1466,589,859,1.600000e-113,410.2,3,2
4,A0A8C0N620,P02460,60.6,259,100,2,1242,1499,602,859,2.200000e-89,330.1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,Q9WTX8,Q9Y6D9,68.4,716,225,1,1,715,1,716,1.600000e-236,817.8,2,2
1264,Q9Y6D9,Q9Y6D9,100.0,718,0,0,1,718,1,718,4.500000e-311,1065.4,2,2
1265,Q9Y3I0,O59245,51.0,386,187,2,121,505,487,871,1.600000e-108,392.1,1,1
1266,Q9Y490,O00291,25.6,328,230,5,2205,2531,696,1010,6.400000e-09,63.5,2,1


In [33]:
groundtruth = np.concatenate([result["GroundTruth"].values,predict_miss.label.values])
predict = np.concatenate([result["Predict"].values,np.zeros(len(predict_miss))])

In [34]:
blast_report = pd.DataFrame(classification_report(groundtruth, predict, zero_division=0, output_dict=True)).T.iloc[1:,]
blast_report.to_csv("blast_report.csv")
blast_report

,precision,recall,f1-score,support
1.0,0.796552,0.082915,0.150195,2786.000000
2.0,0.435216,0.350736,0.388436,747.000000
3.0,0.507042,0.476821,0.491468,151.000000
4.0,0.577320,0.470588,0.518519,119.000000
5.0,0.197183,0.518519,0.285714,27.000000
6.0,0.465517,0.442623,0.453782,61.000000
7.0,0.000000,0.000000,0.000000,7.000000
8.0,0.250000,0.083333,0.125000,12.000000
9.0,0.000000,0.000000,0.000000,4.000000
10.0,1.000000,0.400000,0.571429,5.000000


In [35]:
blast_result = vali_data[["id","label"]]
blast_result["BLAST_Predict"] = predict
blast_result = blast_result[["id","BLAST_Predict"]]
blast_result['BLAST_Predict'] = blast_result['BLAST_Predict'].astype(int)
blast_result

,id,BLAST_Predict
0,Q01147,2
1,Q06507,2
2,P81269,2
3,P97471,3
4,P19123,3
...,...,...
3947,Q9TS33,0
3948,P07293,0
3949,P13806,0
3950,P19517,0


In [36]:
result = pd.merge(output_result,blast_result,left_on = "uniprot_id", right_on = "id")
# result["XGBOOST_Predict"] = result["XGBOOST_Predict"].replace(dict(zip([0,1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8,10,12])))
# result["KNN_Predict"] = result["KNN_Predict"].replace(dict(zip([0,1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8,10,12])))
# result["comple_portal_num"] = result["comple_portal_num"].replace(dict(zip([0,1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8,10,12])))
result = result.rename(columns={'comple_portal_num':'GroundTruth'})
result = result.drop("id",axis=1)
result

,uniprot_id,GroundTruth,XGBOOST_Predict,KNN_Predict,BLAST_Predict
0,Q01147,1,2,2,2
1,Q06507,1,2,2,2
2,P81269,1,2,2,2
3,P97471,1,1,1,3
4,P19123,1,2,1,3
...,...,...,...,...,...
3928,Q9TS33,4,4,2,0
3929,P07293,1,2,4,0
3930,P13806,1,2,2,0
3931,P19517,1,2,2,0


In [37]:
X.shape

(3933, 1280)

In [38]:
from tool.att import Attention
from keras.models import load_model
# deepsub
model = load_model("./model/deepsub.h5",custom_objects={"Attention": Attention},compile=False)
predicted = model.predict(X.reshape(-1,1,1280))
predicted_labels = np.argmax(predicted, axis=1)
label_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12}
y_test_transformed = [label_map[x] for x in predicted_labels]
result["DeepSub_Predict"] = y_test_transformed

123/123 [==============================] - 1s 4ms/step


In [40]:
complex= pd.read_csv('./DATA/complexPortal_num(1).tsv',sep= '\t')
model_location = "./model/QUEEN_MLPmodel_final.pkl"
with open(model_location, "rb") as f:
    QUEEN_model = pickle.load(f)
# Queen pred
y_test = QUEEN_model.predict(X)
inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}

y_test_transformed = np.array([inv_map[x] for x in y_test])
result["Queen_Predict"] = y_test_transformed

In [18]:
result.to_csv("OpenSet_Predict.csv", index=False)

In [44]:
result[(result.GroundTruth==result.DeepSub_Predict)&(result.GroundTruth!=1)&(result.GroundTruth!=result.Queen_Predict)]

,uniprot_id,GroundTruth,XGBOOST_Predict,KNN_Predict,BLAST_Predict,DeepSub_Predict,Queen_Predict
7,P18293,2,2,2,3,2,1
8,P61014,5,5,5,3,5,1
10,P42582,2,2,1,3,2,8
28,Q9CQW5,2,2,2,3,2,1
40,O35930,2,2,2,2,2,1
...,...,...,...,...,...,...,...
3909,E1BMV3,3,1,3,0,3,1
3917,P04258,3,3,3,0,3,6
3923,F1N7F9,2,2,2,0,2,1
3927,P11716,4,4,2,0,4,1
